# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## All CpG

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\LexicalFluency
df = pd.read_csv('LexicalFluency_with_MethylomeDetails.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\LexicalFluency
(233, 85923)


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,Lexical_Fluency_Category,cg02315971,cg24133836,...,cg12092065,cg10388349,cg11918957,cg26583598,cg11023075,cg02097667,cg21616723,cg27335855,cg18259253,cg15404040
0,200973410159_R03C01,3000,2,2,0,69.1,4,1,0.030109,0.016106,...,0.953091,0.392056,0.913402,0.913637,0.897198,0.871237,0.913624,0.651860,0.852542,0.909412
1,200991620021_R01C01,3001,1,1,2,65.1,4,0,0.028697,0.017000,...,0.924513,0.370289,0.902610,0.914253,0.881206,0.834363,0.912120,0.692530,0.839574,0.867794
2,200991620021_R03C01,3002,2,1,2,67.6,4,1,0.025308,0.017806,...,0.921575,0.416247,0.864819,0.942005,0.907732,0.869391,0.907648,0.834695,0.842573,0.917446


In [3]:
df1 = df.drop(['Sentrix','PATNO','HYS','Gender','Lexical_Fluency_Category','Age','AgeCate'], axis=1)
df1.head(2)

,APPRDX,cg02315971,cg24133836,cg03286969,cg11738782,cg25741731,cg24629744,cg23836745,cg11685489,cg06081147,...,cg12092065,cg10388349,cg11918957,cg26583598,cg11023075,cg02097667,cg21616723,cg27335855,cg18259253,cg15404040
0,2,0.030109,0.016106,0.019371,0.884151,0.026326,0.868373,0.905809,0.825562,0.691096,...,0.953091,0.392056,0.913402,0.913637,0.897198,0.871237,0.913624,0.65186,0.852542,0.909412
1,1,0.028697,0.017000,0.018631,0.871526,0.026148,0.866396,0.922285,0.881489,0.647436,...,0.924513,0.370289,0.902610,0.914253,0.881206,0.834363,0.912120,0.69253,0.839574,0.867794


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(60))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.5416666666666667
2
Mean accuracy: 0.525
3
Mean accuracy: 0.4833333333333333
4
Mean accuracy: 0.5166666666666666
5
Mean accuracy: 0.525
6
Mean accuracy: 0.5
7
Mean accuracy: 0.5750000000000001
8
Mean accuracy: 0.5583333333333333
9
Mean accuracy: 0.4916666666666667
10
Mean accuracy: 0.55
11
Mean accuracy: 0.475
12
Mean accuracy: 0.55
13
Mean accuracy: 0.42499999999999993
14
Mean accuracy: 0.475
15
Mean accuracy: 0.5666666666666667
16
Mean accuracy: 0.5000000000000001
17
Mean accuracy: 0.4916666666666667
18
Mean accuracy: 0.4666666666666667
19
Mean accuracy: 0.5666666666666667
20
Mean accuracy: 0.5916666666666667
21
Mean accuracy: 0.5000000000000001
22
Mean accuracy: 0.45
23
Mean accuracy: 0.5833333333333333
24
Mean accuracy: 0.5083333333333334
25
Mean accuracy: 0.4416666666666667
26
Mean accuracy: 0.4333333333333334
27
Mean accuracy: 0.4666666666666668
28
Mean accuracy: 0.475
29
Mean accuracy: 0.475
30
Mean accuracy: 0.4999999999999999
31
Mean accuracy: 0.5666666666666

In [5]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("LexicalFluency_AllCpG_100RF.csv", index = False)

0.5090833333333332


## RanFor 1000 times. Samples = 50

In [6]:
df2 = df1.drop (['APPRDX'], axis=1)

In [7]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(60))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.4666666666666667
2
Mean accuracy: 0.5916666666666668
3
Mean accuracy: 0.5666666666666667
4
Mean accuracy: 0.4833333333333333
5
Mean accuracy: 0.5000000000000001
6
Mean accuracy: 0.4833333333333333
7
Mean accuracy: 0.5916666666666667
8
Mean accuracy: 0.5916666666666666
9
Mean accuracy: 0.5083333333333333
10
Mean accuracy: 0.6
11
Mean accuracy: 0.575
12
Mean accuracy: 0.45
13
Mean accuracy: 0.45
14
Mean accuracy: 0.5833333333333333
15
Mean accuracy: 0.475
16
Mean accuracy: 0.55
17
Mean accuracy: 0.4666666666666667
18
Mean accuracy: 0.4916666666666666
19
Mean accuracy: 0.5416666666666667
20
Mean accuracy: 0.45
21
Mean accuracy: 0.5583333333333332
22
Mean accuracy: 0.5
23
Mean accuracy: 0.525
24
Mean accuracy: 0.4833333333333334
25
Mean accuracy: 0.5166666666666667
26
Mean accuracy: 0.5249999999999999
27
Mean accuracy: 0.5083333333333333
28
Mean accuracy: 0.6000000000000001
29
Mean accuracy: 0.4583333333333333
30
Mean accuracy: 0.6
31
Mean accuracy: 0.5833333333333334
32

In [8]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("LexicalFluency_Rand50CpG_100RF.csv", index = False)

0.5184166666666666


## RanFor 100 times. Samples = 98

In [9]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(98, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(60))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.5166666666666667
2
Mean accuracy: 0.6333333333333333
3
Mean accuracy: 0.4999999999999999
4
Mean accuracy: 0.4666666666666667
5
Mean accuracy: 0.5833333333333333
6
Mean accuracy: 0.4416666666666667
7
Mean accuracy: 0.575
8
Mean accuracy: 0.4916666666666666
9
Mean accuracy: 0.5166666666666666
10
Mean accuracy: 0.5333333333333334
11
Mean accuracy: 0.4833333333333333
12
Mean accuracy: 0.4833333333333334
13
Mean accuracy: 0.5
14
Mean accuracy: 0.5333333333333333
15
Mean accuracy: 0.4833333333333334
16
Mean accuracy: 0.5083333333333333
17
Mean accuracy: 0.45833333333333337
18
Mean accuracy: 0.5916666666666666
19
Mean accuracy: 0.6166666666666667
20
Mean accuracy: 0.4916666666666666
21
Mean accuracy: 0.5333333333333333
22
Mean accuracy: 0.4666666666666667
23
Mean accuracy: 0.5166666666666666
24
Mean accuracy: 0.4666666666666666
25
Mean accuracy: 0.5250000000000001
26
Mean accuracy: 0.4416666666666667
27
Mean accuracy: 0.5
28
Mean accuracy: 0.55
29
Mean accuracy: 0.508333333

In [10]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("LexicalFluency_Rand98CpG_100RF.csv", index = False)

0.5091666666666668


## RanFor Top 50 Boruta 100 times. Samples = 50

In [11]:
df = pd.read_csv('LexicalFluency_Boruta_Top50_Data.csv')
print(df.shape)
df.head(2)

(233, 52)


,PATNO,APPRDX,cg26354221,cg14296039,cg10475683,cg23254463,cg06273051,cg18823302,cg15355235,cg07027506,...,cg13546658,cg15593930,cg13359134,cg07667731,cg09304381,cg23169811,cg26869617,cg21585627,cg02618319,cg09670616
0,3000,2,0.918085,0.873012,0.062533,0.919921,0.543591,0.944782,0.633212,0.645067,...,0.232474,0.019946,0.016097,0.016602,0.981877,0.933661,0.923248,0.042639,0.981089,0.017773
1,3001,1,0.924711,0.835527,0.044298,0.921331,0.492912,0.890833,0.027949,0.578137,...,0.164460,0.027932,0.014686,0.013778,0.960567,0.931264,0.906871,0.057217,0.977055,0.031018


In [12]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg26354221,cg14296039,cg10475683,cg23254463,cg06273051,cg18823302,cg15355235,cg07027506,cg02479278,...,cg13546658,cg15593930,cg13359134,cg07667731,cg09304381,cg23169811,cg26869617,cg21585627,cg02618319,cg09670616
0,2,0.918085,0.873012,0.062533,0.919921,0.543591,0.944782,0.633212,0.645067,0.677445,...,0.232474,0.019946,0.016097,0.016602,0.981877,0.933661,0.923248,0.042639,0.981089,0.017773
1,1,0.924711,0.835527,0.044298,0.921331,0.492912,0.890833,0.027949,0.578137,0.641669,...,0.164460,0.027932,0.014686,0.013778,0.960567,0.931264,0.906871,0.057217,0.977055,0.031018


In [13]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(60))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.6833333333333333
2
Mean accuracy: 0.75
3
Mean accuracy: 0.65
4
Mean accuracy: 0.7333333333333333
5
Mean accuracy: 0.675
6
Mean accuracy: 0.6833333333333332
7
Mean accuracy: 0.7833333333333334
8
Mean accuracy: 0.75
9
Mean accuracy: 0.6333333333333334
10
Mean accuracy: 0.7333333333333334
11
Mean accuracy: 0.725
12
Mean accuracy: 0.7083333333333333
13
Mean accuracy: 0.6416666666666667
14
Mean accuracy: 0.6333333333333333
15
Mean accuracy: 0.7
16
Mean accuracy: 0.6500000000000001
17
Mean accuracy: 0.7166666666666667
18
Mean accuracy: 0.7666666666666666
19
Mean accuracy: 0.8166666666666667
20
Mean accuracy: 0.6499999999999999
21
Mean accuracy: 0.65
22
Mean accuracy: 0.7
23
Mean accuracy: 0.6666666666666666
24
Mean accuracy: 0.7583333333333334
25
Mean accuracy: 0.6500000000000001
26
Mean accuracy: 0.6833333333333333
27
Mean accuracy: 0.7583333333333334
28
Mean accuracy: 0.6666666666666666
29
Mean accuracy: 0.7416666666666667
30
Mean accuracy: 0.675
31
Mean accuracy: 0.6916

In [14]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("LexicalFluency_Boruta50CpG_100RF.csv", index = False)

0.7038333333333334


## RanFor Top 50 NMI 100 times. Samples = 50

In [15]:
df = pd.read_csv('Top50NMI_LexicalFluency_Data.csv')
print(df.shape)
df.head(2)

(233, 52)


,PATNO,APPRDX,cg18504826,cg24706032,cg04958915,cg23983141,cg04310833,cg12184854,cg08550831,cg08096908,...,cg25045630,cg07956247,cg17287767,cg13739582,cg06173919,cg00379002,cg25361663,cg01083159,cg01219549,cg14478495
0,3000,2,0.923461,0.013061,0.953291,0.873134,0.919058,0.935107,0.926300,0.937764,...,0.925686,0.950869,0.027357,0.661946,0.920101,0.070987,0.922851,0.285636,0.978548,0.079537
1,3001,1,0.912524,0.014550,0.912131,0.863465,0.914570,0.933108,0.907085,0.905542,...,0.896759,0.954344,0.030067,0.581498,0.906110,0.081577,0.911043,0.323545,0.975563,0.018922


In [16]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg18504826,cg24706032,cg04958915,cg23983141,cg04310833,cg12184854,cg08550831,cg08096908,cg15355127,...,cg25045630,cg07956247,cg17287767,cg13739582,cg06173919,cg00379002,cg25361663,cg01083159,cg01219549,cg14478495
0,2,0.923461,0.013061,0.953291,0.873134,0.919058,0.935107,0.926300,0.937764,0.685666,...,0.925686,0.950869,0.027357,0.661946,0.920101,0.070987,0.922851,0.285636,0.978548,0.079537
1,1,0.912524,0.014550,0.912131,0.863465,0.914570,0.933108,0.907085,0.905542,0.676244,...,0.896759,0.954344,0.030067,0.581498,0.906110,0.081577,0.911043,0.323545,0.975563,0.018922


In [17]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(60))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.7
2
Mean accuracy: 0.6666666666666666
3
Mean accuracy: 0.6249999999999999
4
Mean accuracy: 0.7083333333333334
5
Mean accuracy: 0.7083333333333334
6
Mean accuracy: 0.675
7
Mean accuracy: 0.725
8
Mean accuracy: 0.725
9
Mean accuracy: 0.7083333333333333
10
Mean accuracy: 0.75
11
Mean accuracy: 0.7
12
Mean accuracy: 0.7666666666666666
13
Mean accuracy: 0.6833333333333333
14
Mean accuracy: 0.725
15
Mean accuracy: 0.8
16
Mean accuracy: 0.6666666666666666
17
Mean accuracy: 0.6666666666666667
18
Mean accuracy: 0.6583333333333334
19
Mean accuracy: 0.7333333333333333
20
Mean accuracy: 0.6833333333333333
21
Mean accuracy: 0.6666666666666667
22
Mean accuracy: 0.6499999999999999
23
Mean accuracy: 0.6916666666666667
24
Mean accuracy: 0.7166666666666667
25
Mean accuracy: 0.7166666666666666
26
Mean accuracy: 0.675
27
Mean accuracy: 0.7166666666666667
28
Mean accuracy: 0.65
29
Mean accuracy: 0.6833333333333333
30
Mean accuracy: 0.6833333333333333
31
Mean accuracy: 0.6666666666666667


In [18]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("LexicalFluency_NMI50CpG_100RF.csv", index = False)

0.7036666666666668


## RanFor Top 50 Boruta + NMI 100 times. Samples = 98

In [19]:
df = pd.read_csv('LexicalFluency_Top50sMerge_Data.csv')
print(df.shape)
df.head(2)

(233, 100)


,PATNO,APPRDX,cg18504826,cg24706032,cg04958915,cg23983141,cg04310833,cg12184854,cg08550831,cg08096908,...,cg13546658,cg15593930,cg13359134,cg07667731,cg09304381,cg23169811,cg26869617,cg21585627,cg02618319,cg09670616
0,3000,2,0.923461,0.013061,0.953291,0.873134,0.919058,0.935107,0.926300,0.937764,...,0.232474,0.019946,0.016097,0.016602,0.981877,0.933661,0.923248,0.042639,0.981089,0.017773
1,3001,1,0.912524,0.014550,0.912131,0.863465,0.914570,0.933108,0.907085,0.905542,...,0.164460,0.027932,0.014686,0.013778,0.960567,0.931264,0.906871,0.057217,0.977055,0.031018


In [20]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg18504826,cg24706032,cg04958915,cg23983141,cg04310833,cg12184854,cg08550831,cg08096908,cg15355127,...,cg13546658,cg15593930,cg13359134,cg07667731,cg09304381,cg23169811,cg26869617,cg21585627,cg02618319,cg09670616
0,2,0.923461,0.013061,0.953291,0.873134,0.919058,0.935107,0.926300,0.937764,0.685666,...,0.232474,0.019946,0.016097,0.016602,0.981877,0.933661,0.923248,0.042639,0.981089,0.017773
1,1,0.912524,0.014550,0.912131,0.863465,0.914570,0.933108,0.907085,0.905542,0.676244,...,0.164460,0.027932,0.014686,0.013778,0.960567,0.931264,0.906871,0.057217,0.977055,0.031018


In [21]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(60))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.7083333333333334
2
Mean accuracy: 0.725
3
Mean accuracy: 0.6333333333333333
4
Mean accuracy: 0.7583333333333333
5
Mean accuracy: 0.7166666666666667
6
Mean accuracy: 0.6666666666666666
7
Mean accuracy: 0.7666666666666666
8
Mean accuracy: 0.725
9
Mean accuracy: 0.725
10
Mean accuracy: 0.7416666666666667
11
Mean accuracy: 0.6500000000000001
12
Mean accuracy: 0.7583333333333334
13
Mean accuracy: 0.6416666666666667
14
Mean accuracy: 0.7
15
Mean accuracy: 0.7583333333333333
16
Mean accuracy: 0.6333333333333333
17
Mean accuracy: 0.7166666666666667
18
Mean accuracy: 0.6916666666666667
19
Mean accuracy: 0.7833333333333333
20
Mean accuracy: 0.6916666666666667
21
Mean accuracy: 0.675
22
Mean accuracy: 0.675
23
Mean accuracy: 0.7000000000000001
24
Mean accuracy: 0.7333333333333334
25
Mean accuracy: 0.7083333333333334
26
Mean accuracy: 0.7083333333333333
27
Mean accuracy: 0.7416666666666667
28
Mean accuracy: 0.675
29
Mean accuracy: 0.7166666666666666
30
Mean accuracy: 0.675
31
Me

In [22]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("LexicalFluency_Top50sCpG98_100RF.csv", index = False)

0.7075833333333336
